In [2]:
# import dependencies
import requests
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# connect to API key
from config import gkey


In [3]:
# read zip codes csv
zipcodes_data = "short_zips.csv" 

# creata a dataframe with the zip codes
zipcodes_df = pd.read_csv(zipcodes_data)
zipcodes_df


,Zip Code
0,10001
1,10002
2,10003
3,10004
4,10005
5,10006
6,10007
7,10008


In [4]:
# Add columns for income, # single, # no cards
zipcodes_df["Avg Income ($)"] = ""
zipcodes_df["# Single"] = ""
zipcodes_df["# No Cars"] = ""
zipcodes_df


,Zip Code,Avg Income ($),# Single,# No Cars
0,10001,,,
1,10002,,,
2,10003,,,
3,10004,,,
4,10005,,,
5,10006,,,
6,10007,,,
7,10008,,,


In [28]:
# establish base URL 
base_url = "https://api.census.gov/data/2017/acs/acs5/profile"

# print the response to see if we have a successful connection
response = requests.get(base_url)
print(response)


<Response [200]>


In [12]:
# loop through all zips in the df, adding the census variables to each row

for index, row in zipcodes_df.iterrows():    
    
    zip = row['Zip Code']
    
    query_url = f"{base_url}?get={avg_income},{single_hh},{hh_no_vehicles},NAME&for=zip%20code%20tabulation%20area:{zip}&key={gkey}"
    
    response = requests.get(query_url).json()
    #pprint(response)
    
    try:
        print(f"Finding demo data for zip code: {zip}")
        
        zipcodes_df.loc[index, "Avg Income ($)"] = response[1][0]
        zipcodes_df.loc[index, "# Single"] = response[1][1]
        zipcodes_df.loc[index, "# No Cars"] = response[1][2]
        #zipcodes_df.loc[index, "# Chinese"] = response[1][3]

        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-------------------------------------------")
    
    
 # !!!! notice the loop dies at zip 10008, despite error handling !!!!!!!   

Finding demo data for zip code: 10001
-------------------------------------------
Finding demo data for zip code: 10002
-------------------------------------------
Finding demo data for zip code: 10003
-------------------------------------------
Finding demo data for zip code: 10004
-------------------------------------------
Finding demo data for zip code: 10005
-------------------------------------------
Finding demo data for zip code: 10006
-------------------------------------------
Finding demo data for zip code: 10007
-------------------------------------------


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
# output data frame with info added in new columns
zipcodes_df


,Zip Code,Avg Income ($),# Single,# No Cars
0,10001,161227,6554,10193
1,10002,65718,14219,27306
2,10003,182531,15067,20895
3,10004,191760,808,1287
4,10005,201734,1728,3538
5,10006,196771,881,1527
6,10007,376573,1029,1961
7,10008,,,


In [ ]:
#  !!!!!!  from this approach we can see that the census API DOES have data for some of the
#  zip codes that were't found when we took the first approach (zip 10007, for example) !!!!!
